In [1]:
import pandas as pd
import gc

In [2]:
dtypes = {
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'hour'          : 'uint8',
        'is_attributed' : 'uint8', 
        'ip_day_hour_count': 'uint32', 
        'ip_os_day_hour_count': 'uint32', 
        'ip_app_day_hour_count': 'uint32', 
        'ip_app_os_day_hour_count': 'uint32', 
        'app_day_hour_count': 'uint32', 
        'ip_device_os_count': 'uint32', 
        'ip_app_device_os_count': 'uint32', 
        'ip_device_os_mean': 'float16',
        'ip_app_device_os_mean': 'float16',
        'ip_app_device_mean': 'float16',
        'app_device_os_mean': 'float16',
        'ip_device_os_time2nextclick': 'int32',
        'ip_app_device_os_time2nextclick': 'int32',
        'ip_app_device_time2nextclick': 'int32',
        'ip_device_os_time2previousclick': 'int32',
        'ip_app_device_os_time2previousclick': 'int32',
        'ip_app_device_time2previousclick': 'int32',
        'ip_device_os_countfromfuture': 'uint32', 
        'ip_app_device_os_countfromfuture': 'uint32', 
        'ip_app_device_countfromfuture': 'uint32', 
        'ip_device_os_countfrompast': 'uint32', 
        'ip_app_device_os_countfrompast': 'uint32', 
        'ip_app_device_countfrompast': 'uint32', 
        'ip_device_os_lasttimediff': 'int32',
        'ip_app_device_os_lasttimediff': 'int32',
        'ip_app_device_lasttimediff': 'int32',
        'ip_device_os_firsttimediff': 'int32',
        'ip_app_device_os_firsttimediff': 'int32',
        'ip_app_device_firsttimediff': 'int32',
        'matrixFact_user_iposdeviceapp_item_app': 'float64',#'float16',
        'matrixFact_user_ip_item_appdeviceos': 'float64',
        'matrixFact_user_ipchannel_item_appdeviceos': 'float64',
        'ip_device_os_regression': 'float64',
        'ip_app_device_os_regression': 'float64',
        'ip_app_device_regression': 'float64',
        'ip_app_device_os_channel_regression': 'float64'
        } 

load_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/'
file_format = '{}_equalhour_supplementV1.csv'
# ALL_features_supplementV3_feature42.csv
# test_features_supplementV3_feature42.csv

#day7 = pd.read_csv(load_path+file_format.format('day7'),dtype=dtypes)
#print('load day7 done!')
day9 = pd.read_csv(load_path+file_format.format('day9'),dtype=dtypes)
print('load day9 done!')
day8 = pd.read_csv(load_path+file_format.format('day8'),dtype=dtypes)
print('load day8 done!')
day7 = pd.read_csv(load_path+file_format.format('day7'),dtype=dtypes)
print('load day7 done!')
day10_test = pd.read_csv(load_path+file_format.format('test'),dtype=dtypes)
print('load day10_test done!')

In [5]:
load_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/'
alltrain = pd.read_csv(load_path+'ALL_features_supplementV3_feature42.csv', dtype=dtypes)
print('load all train done!')

test = pd.read_csv(load_path+'test_features_supplementV3_feature42.csv',dtype=dtypes)

test.drop(list(set(test.columns) - set(alltrain.columns)), axis=1, inplace=True)

print(set(test.columns) - set(alltrain.columns))

load all train done!


In [17]:
test.shape

(18790469, 39)

In [18]:
alltrain.shape

(184903890, 40)

ori_path = '/home/kai/data/kaggle/talkingdata/data/'

test_ori = pd.read_csv(ori_path+"test.csv", dtype=dtypes, usecols=['ip','app','device','os', 'channel', 'click_time', 'click_id'])

In [7]:
day9.shape, day8.shape, day7.shape, day10_test.shape

((20898422, 49), (20446743, 49), (19534560, 49), (18790469, 48))

day9len = 20898422
day8len = 20446743
day7len = 19534560
day10_testlen = 18790469

train_df = day9.append(day8)
train_df = train_df.append(day7)
train_df = train_df.append(day10_test)

del day9, day8, day7, day10_test; gc.collect()

In [21]:
train_len = len(alltrain)
test_len = len(test)
train_len, test_len

(184903890, 18790469)

In [22]:
alltrain = alltrain.append(test)
alltrain.shape

(203694359, 40)

In [25]:
train_df = alltrain

In [23]:
del test; gc.collect()

587

In [26]:
drop_cols = [col for col in train_df.columns if 'countself' in col or 'regression' in col]
print(drop_cols)

['ip_app_device_os_channel_regression']


In [27]:
train_df.drop(drop_cols, axis=1, inplace=True)

In [28]:
train_df.shape

(203694359, 39)

In [30]:
mean_cols = [col for col in train_df.columns if '_mean' in col]

print(mean_cols)

for col in mean_cols:
    train_df[col] = train_df[col].apply(lambda x: 1000*(x+0.02))

['app_device_os_mean', 'ip_app_device_mean', 'ip_app_device_os_mean', 'ip_device_os_mean']


In [32]:
import sys

sys.getsizeof(train_df)/1024**3

In [35]:
embedding_features = ['app','channel','device','os','hour']

numeric_features = list(set(set(train_df.columns) - set(embedding_features)) - set(['is_attributed']))
print(numeric_features, len(numeric_features))

['ip_app_device_os_mean', 'ip_app_device_os_countfrompast', 'ip_app_device_countfrompast', 'ip_app_device_os_lasttimediff', 'ip_app_device_time2previousclick', 'app_device_os_mean', 'ip_app_device_os_count', 'ip_device_os_countfrompast', 'ip_device_os_time2previousclick', 'ip_app_device_lasttimediff', 'ip_os_day_hour_count', 'ip_app_device_firsttimediff', 'ip_app_day_hour_count', 'ip_device_os_firsttimediff', 'ip_device_os_count', 'ip_app_device_os_time2nextclick', 'ip_device_os_lasttimediff', 'ip_app_device_time2nextclick', 'ip_day_hour_count', 'ip_device_os_mean', 'matrixFact_user_ip_item_appdeviceos', 'ip_app_device_mean', 'ip_app_device_os_time2previousclick', 'ip_app_os_day_hour_count', 'ip_device_os_countfromfuture', 'attributed_timediff', 'app_day_hour_count', 'ip_app_device_countfromfuture', 'matrixFact_user_ipchannel_item_appdeviceos', 'ip_app_device_os_countfromfuture', 'ip_app_device_os_firsttimediff', 'ip_device_os_time2nextclick', 'matrixFact_user_iposdeviceapp_item_app'] 

In [36]:
from sklearn.preprocessing import LabelEncoder
train_df[embedding_features] = train_df[embedding_features].apply(LabelEncoder().fit_transform)

In [37]:
max_dict = dict(train_df[embedding_features].max())

In [38]:
max_dict

{'app': 729, 'channel': 201, 'device': 3798, 'hour': 23, 'os': 855}

{'app': 729, 'channel': 201, 'device': 3798, 'hour': 23, 'os': 855}

for col in train.columns:
    nan_amount = len(np.argwhere(np.isnan(day8[col])))
    if nan_amount != 0:
        print('train', col, nan_amount)
    nan_amount = len(np.argwhere(np.isnan(day9[col])))
    if nan_amount != 0:
        print('test', col, nan_amount)

In [41]:
len(numeric_features)

33

In [42]:
len(train_df)

203694359

In [43]:
train_len, -train_len

(184903890, -184903890)

In [46]:
train = train_df[:train_len]#(day9len+day8len+day7len)]
test = train_df[-test_len:]#day10_testlen:]

assert len(train) + len(test) == len(train_df)
assert len(test) == test_len

In [47]:
train.shape, test.shape

((184903890, 39), (18790469, 39))

In [50]:
del train_df; gc.collect()

1197

In [51]:
from sklearn.preprocessing import StandardScaler

print('fitting...')
scaler_dict = dict((col, StandardScaler().fit(train[[col]].values)) for col in numeric_features)

fitting...


/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint32 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int32 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [52]:
import multiprocessing
import numpy as np
from tqdm import tqdm
workers = 20

def _apply_df(args):
    df, func, kwargs = args
    return df.apply(func, **kwargs)

def apply_by_multiprocessing(df, func, **kwargs):
    workers = kwargs.pop('workers')
    pool = multiprocessing.Pool(processes=workers)
    result = pool.map(_apply_df, [(d, func, kwargs)
            for d in np.array_split(df, workers)])
    pool.close()
    return pd.concat(list(result))

def scaler_transform(x, **kwargs):
    scaler_dict=kwargs.pop('scaler_dict')
    col = kwargs.pop('col')
    return scaler_dict[col].transform(x)[0][0]

In [53]:
gc.collect()

1222

In [55]:
for col in tqdm(numeric_features):
    train[col]=apply_by_multiprocessing(train[col], scaler_transform, workers=workers, col=col, scaler_dict=scaler_dict)
    test[col]=apply_by_multiprocessing(test[col], scaler_transform, workers=workers, col=col, scaler_dict=scaler_dict)

  0%|          | 0/33 [00:00<?, ?it/s]/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
100%|██████████| 33/33 [3:03:50<00:00, 334.26s/it]  


ss_numeric_features = ['ss_'+col for col in number_features]

In [73]:
train['ip_device_os_lasttimediff'].std(), test['ip_device_os_lasttimediff'].std()

(1.0000000027041074, 0.6089447229927581)

#print('transforming...')
for col in tqdm(number_features):
    train[col] = train[col].apply(lambda x: scaler_dict[col].transform(x)[0][0])
    test[col] = test[col].apply(lambda x: scaler_dict[col].transform(x)[0][0])

In [66]:
from keras.models import Model
from keras.layers import Dense, Input, Embedding, Dropout, Activation, Reshape, Flatten
from keras.layers.merge import concatenate, dot, add, multiply
from keras.layers.normalization import BatchNormalization
from keras.layers import SpatialDropout1D, Conv1D
from keras.layers.advanced_activations import LeakyReLU, PReLU, ELU
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.regularizers import l1, l2, l1_l2
from keras.initializers import RandomUniform
from keras.optimizers import RMSprop, Adam, SGD

import numpy as np

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [67]:
def get_keras_data(dataset):
    X = dict((col, np.array(dataset[col])) for col in embedding_features)
    X_num =  dict(('num_'+col, np.array(dataset[col])) for col in numeric_features)
    X.update(X_num)
    return X

#fe = concatenate([(emb_) for emb_ in emb_model.values()])
# Rest of the model
#model = Model(inputs=[inp for inp in emb_inputs.values()] + [inp for inp in num_inputs.values()], outputs=outp)

In [168]:
import gc; gc.collect()

1892

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

In [ ]:
((20898422, 49), (20446743, 49), (19534560, 49), (18790469, 48))

In [171]:
train.shape, test.shape

((60879725, 70), (18790469, 70))

In [245]:
del train_df, y_train, val_df, y_val; gc.collect()

924

In [189]:
val_size = 5000000

In [195]:
del train_df, y_train, val_df, y_val; gc.collect()

12911

In [235]:
train.shape

(60879725, 70)

# SAVE

train.reset_index().to_feather('Feathers/ss_alltrain_typechanged.ftr')

test.reset_index().to_feather('Feathers/ss_test_via_alltrain_typechanged.ftr')

# LOAD

In [248]:
del test_s.index.name

In [222]:
test_load = pd.read_feather('Feathers/ss_test_via_alltrain_typechanged.ftr')

In [224]:
test_load.shape

(18790469, 40)

In [226]:
test_load.set_index('index').

,app,app_day_hour_count,app_device_os_mean,attributed_timediff,channel,device,hour,ip_app_day_hour_count,ip_app_device_countfromfuture,ip_app_device_countfrompast,...,ip_device_os_lasttimediff,ip_device_os_mean,ip_device_os_time2nextclick,ip_device_os_time2previousclick,ip_os_day_hour_count,is_attributed,matrixFact_user_ip_item_appdeviceos,matrixFact_user_ipchannel_item_appdeviceos,matrixFact_user_iposdeviceapp_item_app,os
index,,,,,,,,,,,,,,,,,,,,,
0,9,0.790197,-0.075500,0.091585,17,1,4,-0.215741,-0.197744,-0.192336,...,-1.132750,0.008702,-0.205401,-0.205966,-0.217264,NaN,0.521780,0.467860,1.140992,3
1,9,0.790197,-0.075500,-0.343579,177,1,4,-0.119355,-0.060634,-0.101353,...,0.553337,0.201479,-0.204271,-0.205401,-0.190076,NaN,0.239275,0.255219,0.778947,3
2,21,-1.007531,-0.089199,-1.370941,34,1,4,-0.204267,-0.050136,-0.196789,...,0.610330,0.167155,-0.188451,-0.172631,-0.089094,NaN,-1.008456,-0.871535,-0.609683,19
3,15,-0.410553,-0.085623,-1.546604,20,1,4,-0.211152,-0.192336,-0.199016,...,0.562604,0.201899,-0.205683,-0.206813,-0.032777,NaN,0.166295,-0.025859,-0.519172,13
4,12,0.431785,-0.089174,-1.257825,112,1,4,-0.218036,-0.200925,-0.204424,...,0.546376,0.008702,-0.205966,-0.204836,-0.170657,NaN,-0.335152,-0.104071,0.062004,13


In [58]:
gc.collect()

14

In [257]:
test_ftr = pd.read_feather('Feathers/standardscaled_test_inorder_no_clickid.ftr')

In [59]:
df_hour = pd.read_csv('/home/kai/data/kaggle/talkingdata/data/hourdistri.csv', index_col='Unnamed: 0')
index = {}
for day in ['day7', 'day8','day9']:
    index[day] = list(range(df_hour.loc[day,'4start'], df_hour.loc[day,'6end0sec'])) + \
    list(range(df_hour.loc[day,'9start'], df_hour.loc[day,'11end0sec'])) + \
    list(range(df_hour.loc[day,'13start'], df_hour.loc[day,'15end0sec'])) 

In [91]:
train_old89 = pd.read_feather('Feathers/standardscaled_train_day9_20898422_day8_20446743_day7_19534560.ftr')

In [ ]:
count_cols = [col for col in numeric_features if 'count' in col]

test_old = pd.read_feather('Feathers/standardscaled_test_inorder_no_clickid.ftr')

In [176]:
alltrain_len = len(train)

184903890

In [177]:
allindexintrain = set(range(alltrain_len))

notday7eh_idx = list(allindexintrain - set(index['day7']))

assert len(notday7eh_idx) == 165369330

del allindexintrain; gc.collect()

In [100]:
train_old89['ss_ip_app_device_os_mean'].mean()

8.596765608504827e-17

In [101]:
temp['ip_app_device_os_mean'].mean()

0.005748842014999835

In [175]:
day8start = int(len(train)*1/3)
day8start

61634630

In [116]:
day9start = int(len(train)*2/3)
day9start

123269260

In [ ]:
del train_df, y_train, val_df, y_val; gc.collect()

In [208]:
import sys

sys.getsizeof(train)/1024**3

32.208416391164064

In [195]:
train['app'].dtype == np.int64

True

In [218]:
gc.collect()

429

for col in numeric_features:
    test[col] = test[col].astype(np.float32)

In [202]:
np.iinfo(np.int16).min, np.iinfo(np.int16).max

(-32768, 32767)

In [213]:
test.dtypes

app                                             int16
app_day_hour_count                            float32
app_device_os_mean                            float32
attributed_timediff                           float32
channel                                         int16
device                                          int16
hour                                            int16
ip_app_day_hour_count                         float32
ip_app_device_countfromfuture                 float32
ip_app_device_countfrompast                   float32
ip_app_device_firsttimediff                   float32
ip_app_device_lasttimediff                    float32
ip_app_device_mean                            float32
ip_app_device_os_count                        float32
ip_app_device_os_countfromfuture              float32
ip_app_device_os_countfrompast                float32
ip_app_device_os_firsttimediff                float32
ip_app_device_os_lasttimediff                 float32
ip_app_device_os_mean       

In [186]:
train_df = get_keras_data(train.iloc[notday7eh_idx])#[day8start:])#train.iloc[index['day8']+index['day9']])#:(day9len+day8len)])
y_train = train.iloc[notday7eh_idx]['is_attributed'].values

len(y_train), len(train_df['app'])

KeyboardInterrupt: 

In [119]:
val_df = get_keras_data(train.iloc[index['day7']])#[(day9len+day8len):(day9len+day8len+day7len)])
y_val = train.iloc[index['day7']]['is_attributed'].values

len(y_val), len(val_df['app'])

(19534560, 19534560)

In [222]:
test_df = get_keras_data(test)

len(test_df), len(test_df['app'])

In [135]:
from sklearn.metrics import roc_auc_score
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1, batch_size=50000):
        super(Callback, self).__init__()
        print('RocAuc evaluating batch size is: {}'.format(batch_size))
        self.interval = interval
        self.X_val, self.y_val = validation_data
        self.batch_size = batch_size

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            print('\non epoch end, start predicting validation set...')
            y_pred = self.model.predict(self.X_val, batch_size=self.batch_size,verbose=1)
            print('start calculating ROC-AUC...')
            score = roc_auc_score(self.y_val, y_pred)
            print("ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))
            
            
import keras
class RocAucMetricCallback(keras.callbacks.Callback):
    def __init__(self, validation_data=(), predict_batch_size=100000, include_on_batch=False):
        super(RocAucMetricCallback, self).__init__()
        self.predict_batch_size=predict_batch_size
        self.include_on_batch=include_on_batch
        self.X_val, self.y_val = validation_data

    def on_batch_begin(self, batch, logs={}):
        pass

    def on_batch_end(self, batch, logs={}):
        if(self.include_on_batch):
            logs['roc_auc_val']=float('-inf')
            if(self.validation_data):
                logs['roc_auc_val']=roc_auc_score(self.y_val,
                                              self.model.predict(self.X_val,
                                                                 batch_size=self.predict_batch_size))
                
    def on_train_begin(self, logs={}):
        if not ('roc_auc_val' in self.params['metrics']):
            self.params['metrics'].append('roc_auc_val')

    def on_train_end(self, logs={}):
        pass

    def on_epoch_begin(self, epoch, logs={}):
        pass

    def on_epoch_end(self, epoch, logs={}):
        logs['roc_auc_val']=float('-inf')
#         y_val = self.validation_data[-3]
#         val_df = self.validation_data[:-3]
        if(self.validation_data):
            logs['roc_auc_val']=roc_auc_score(self.y_val,
                                              self.model.predict(self.X_val,
                                                                 batch_size=self.predict_batch_size))
#             logs['roc_auc_val']=roc_auc_score(y_val,#self.validation_data[1], 
#                                               self.model.predict(val_df,#self.validation_data[0],
#                                                                  batch_size=self.predict_batch_size))

In [173]:
#embids = embedding_features#['app', 'channel', 'device', 'os', 'hour', 'day', 'wday', 'qty', 'ip_app_count', 'ip_app_os_count']
# get the max of each code type
embmaxs = dict((col, max_dict[col]+1) for col in embedding_features)

emb_n = 50

# Build the inputs, embeddings and concatenate them all for each column
emb_inputs = dict((col, Input(shape=[1], name = col))  for col in embedding_features)
emb_model  = dict((col, Embedding(embmaxs[col], emb_n)(emb_inputs[col])) for col in embedding_features)

fe = concatenate([(emb_) for emb_ in emb_model.values()])

s_dout = SpatialDropout1D(0.2)(fe)

conv = Conv1D(100, kernel_size=4, strides=1, padding='same')(s_dout)

num_inputs = dict((col, Input(shape=[1], name = 'num_'+col))  for col in numeric_features)

num_fe = concatenate([(num_) for num_ in num_inputs.values()])

concat = concatenate([Flatten()(s_dout), Flatten()(conv), (num_fe)])

dense_n = 1000

x = Dropout(0.2)(Dense(dense_n,activation='relu')(concat))
x = Dropout(0.2)(Dense(dense_n,activation='relu')(x))

outp = Dense(1,activation='sigmoid')(x)

model = Model(inputs=[inp for inp in emb_inputs.values()] + [inp for inp in num_inputs.values()], outputs=outp)
#model = Model(inputs=[inp for inp in emb_inputs.values()], outputs=outp)

batch_size = 200000
epochs = 12
exp_decay = lambda init, fin, steps: (init/fin)**(1/(steps-1)) - 1
steps = int(len(list(train_df)[0]) / batch_size) * epochs
lr_init, lr_fin = 0.002, 0.0002
lr_decay = exp_decay(lr_init, lr_fin, steps)
optimizer_adam = Adam(lr=0.002, decay=lr_decay)
model.compile(loss='binary_crossentropy',optimizer=optimizer_adam,metrics=['accuracy'])


# checkpoint = ModelCheckpoint(model_save_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
# earlystopping = EarlyStopping(monitor='val_loss', mode="min", patience=8)
# rocauc = RocAucEvaluation(validation_data=(val_df, y_val), interval=1, batch_size=pred_batch_size)

import time
tempid = str(int(time.time()))[3:]
print('Model ID: ' + tempid) 
seed = 7
np.random.seed(seed)
pred_batch_size = 100000
model_save_path = 'NN_MODELS/NN_data{}_batchsize{}_ep{}_{}.hdf5'.format(len(y_train), batch_size, epochs, tempid)

cb = [
    RocAucMetricCallback(validation_data=(val_df, y_val)), # include it before EarlyStopping!
    EarlyStopping(monitor='roc_auc_val',mode='max',patience=1, verbose=2),
    ModelCheckpoint(model_save_path, monitor='roc_auc_val', verbose=1, save_best_only=True, mode='max'),
#     RocAucEvaluation(validation_data=(val_df, y_val), interval=1, batch_size=pred_batch_size)
]

class_weight = {0:1,1:99}

model.fit(train_df, y_train, batch_size=batch_size, epochs=epochs, 
          validation_data=(val_df, y_val), 
#           validation_split = 0.5,
          class_weight=class_weight,
          shuffle=True, verbose=1, callbacks=cb)#[rocauc, earlystopping, checkpoint])

Model ID: 5376780
Train on 1000000 samples, validate on 1000002 samples
Epoch 1/12
 800000/1000000 [=======================>......] - ETA: 1s - loss: 0.5338 - acc: 0.7740
Epoch 00001: roc_auc_val improved from -inf to 0.95180, saving model to NN_data1000000_batchsize200000_ep12_5376780.hdf5
1000000/1000000 [==============================] - 11s 11us/step - loss: 0.4664 - acc: 0.8126 - val_loss: 0.1507 - val_acc: 0.9507 - roc_auc_val: 0.9518
Epoch 2/12
 800000/1000000 [=======================>......] - ETA: 0s - loss: 0.2146 - acc: 0.9687
Epoch 00002: roc_auc_val improved from 0.95180 to 0.96111, saving model to NN_data1000000_batchsize200000_ep12_5376780.hdf5
1000000/1000000 [==============================] - 8s 8us/step - loss: 0.2113 - acc: 0.9729 - val_loss: 0.0601 - val_acc: 0.9905 - roc_auc_val: 0.9611
Epoch 3/12
 800000/1000000 [=======================>......] - ETA: 0s - loss: 0.1836 - acc: 0.9920
Epoch 00003: roc_auc_val did not improve
1000000/1000000 [========================

In [147]:
temp = list((val_df, y_val))

In [148]:
type(temp)

list

In [155]:
len(temp)

2

In [149]:
temp

[{'app': array([25,  2]),
  'channel': array([91, 80]),
  'device': array([1, 1]),
  'hour': array([16, 16]),
  'num_app_day_hour_count': array([-1.19482976, -0.06755558]),
  'num_app_device_os_mean': array([-0.09337464, -0.08192172]),
  'num_attributed_timediff': array([1.99193876, 0.16344757]),
  'num_ip_app_day_hour_count': array([-0.2203312 ,  0.42683637]),
  'num_ip_app_device_countfromfuture': array([-0.196153  ,  3.30188843]),
  'num_ip_app_device_countfrompast': array([-0.21333145, -0.16561353]),
  'num_ip_app_device_firsttimediff': array([-1.81162421, -1.75880579]),
  'num_ip_app_device_lasttimediff': array([2.15815736, 2.31732789]),
  'num_ip_app_device_mean': array([0.00547391, 0.0118192 ]),
  'num_ip_app_device_os_count': array([-0.17801885,  1.41273736]),
  'num_ip_app_device_os_countfromfuture': array([-0.15549682,  4.78392936]),
  'num_ip_app_device_os_countfrompast': array([-0.16124257, -0.08463263]),
  'num_ip_app_device_os_firsttimediff': array([-1.06129982, -1.012240

In [139]:
len(val_df)

38

In [143]:
y_val

array([0., 0.])

In [227]:
model.load_weights('NN_batchsize200000_ep8_5308912.hdf5')

In [228]:
val_pred_savedmodel = model.predict(val_df, batch_size=pred_batch_size, verbose=1)

19534560/19534560 [==============================] - 42s 2us/step


In [203]:
roc_auc_score(y_val, val_pred_fullmodel)

0.9817132065702936

In [230]:
roc_auc_score(y_val, val_pred_savedmodel)

0.9818143556926712

In [231]:
sub = pd.DataFrame()
sub['click_id'] = test_ori['click_id'].astype('int')

print("predicting....")
sub['is_attributed'] = model.predict(test_df, batch_size=pred_batch_size, verbose=1)

flag = 'saved'
sub.to_csv('nn_{}.csv.gz'.format(flag, tempid), index=False, float_format='%.9f', compression='gzip')

predicting....
18790469/18790469 [==============================] - 41s 2us/step


In [ ]:
# model predict val
# model predict test

# THEN load saved model 
# model predict val
# model predict test

In [66]:
roc_auc_score(y_val_1, val_pred_1) # 0.01

0.9788076360576548

In [55]:
roc_auc_score(y_val_1, val_pred_1) # 0.001

0.9761119662504989

In [39]:
embedding_inputs = []
embedding_outputs = []

In [43]:
output_dim = 50
for i in range(len(embedding_features)):
    tmp_input = Input(shape=(1,), dtype='int32', name=embedding_features[i]+'_input')
    tmp_embeddings = Embedding(int(train_embeddings[i].max()+1),output_dim,name=embedding_features[i]+'_embeddings')(tmp_input)
    tmp_embeddings = Flatten(name=embedding_features[i]+'_flatten')(tmp_embeddings)
    
    embedding_inputs.append(tmp_input)
    embedding_outputs.append(tmp_embeddings)

In [53]:
embedding_outputs.append(number_input)

In [57]:
profile = concatenate(embedding_outputs, name='profile')

In [58]:
profile

<tf.Tensor 'profile/concat:0' shape=(?, ?) dtype=float32>

In [59]:
def FunctionalDense(n, x, batchnorm=False, act='relu', lw1=0.0, dropout=0, name=''):
    if lw1 == 0.0:
        x = Dense(n, name=name+'_dense')(x)
    else:
        x = Dense(n, kernel_regularizer=l1(lw1), name=name+'_dense')(x)
    
    if batchnorm:
        x = BatchNormalization(name=name+'_batchnorm')(x)
        
    if act in {'relu', 'tanh', 'sigmoid'}:
        x = Activation(act, name=name+'_activation')(x)
    elif act =='prelu':
        x = PReLU(name=name+'_activation')(x)
    elif act == 'leakyrelu':
        x = LeakyReLU(name=name+'_activation')(x)
    elif act == 'elu':
        x = ELU(name=name+'_activation')(x)
    
    if dropout > 0:
        x = Dropout(dropout, name=name+'_dropout')(x)
        
    return x

In [62]:
profile_embedddings = FunctionalDense(100, profile, batchnorm=True, dropout=0.2, name='profile')

In [47]:
import numpy as np

In [42]:
drop_out = np.random.randint(2,5)/10
drop_out

0.4

In [ ]:
emb_n = np.random.randint(30, 60)
activation = np.random.choice(['relu', 'tanh', 'prelu', 'leakyrelu', 'elu'])
batchnorm = np.random.choice([True, False])
dense_n = np.random.randint(100, 500)
class_weight = np.random.randint(70, 400)
lr = 7.5e-3 * (0.1 ** (np.random.rand() * 2 - 1.0))
lr_decay = 0.65 + np.random.rand() * 0.3

In [28]:
lw = 7.5e-4 * (0.1 ** (np.random.rand() * 3 - 1.5))
lw

0.0022744712879675427

In [18]:
{0:1,1:np.random.randint(70, 400)}

{0: 1, 1: 166}

In [15]:

class_weight

292

In [ ]:
emb_trainable: true

emb_n: 30 60
s_dout_drop: 0.1 0.5
conv_dim: 50 150
conv_kernel_size: 3 8
dense_n: 50 500
batchnorm: t f
act: relu prelu leakyrelu elu
preds_drop: 0.2 0.5
class_weight: 99 400
lr
lr_decay

In [23]:

lr, lr_decay

(0.004787722047296411, 0.7955989835149024)

In [6]:
import pandas as pd

In [7]:
nn_record = pd.read_csv('nn_record.csv', 
                        names=['tempid', 'emb_n', 'lr', 'lr_decay', 'conv_n', 'dense_n', \
                               'drop_out', 'act', 'batchnorm', 'pos_weight', 'NA1','NA2','seed', \
                               'time','mode', 'bst_epoch', 'trn_loss', 'trn_acc', 'val_loss', 'val_acc', 'val_auc']).sort_values(by='tempid', ascending=False)

In [8]:
nn_record

,tempid,emb_n,lr,lr_decay,conv_n,dense_n,drop_out,act,batchnorm,pos_weight,...,NA2,seed,time,mode,bst_epoch,trn_loss,trn_acc,val_loss,val_acc,val_auc
41,6044457,71,0.015905,0.928064,121,44,0.1,relu,False,170,...,-1,23694852,2018-05-11 09:43:17,1,1,0.36491,0.97834,0.06364,0.98375,0.98126
40,6035465,49,0.017287,0.726490,61,67,0.1,leakyrelu,False,314,...,-1,33732111,2018-05-11 09:13:44,1,14,0.26897,0.96680,0.11165,0.96874,0.98372
39,6029468,77,0.002535,0.663915,108,53,0.3,elu,True,91,...,-1,7018820,2018-05-11 06:43:50,1,8,0.12703,0.98749,0.04703,0.98809,0.98474
38,6027720,47,0.023634,0.944210,133,52,0.2,relu,False,240,...,-1,23815266,2018-05-11 05:03:55,1,1,0.45366,0.96714,0.16182,0.95860,0.97994
37,6022902,66,0.021590,0.910522,80,33,0.2,leakyrelu,True,160,...,-1,144700,2018-05-11 04:34:45,1,6,0.18651,0.98084,0.08905,0.97726,0.98367
36,6015619,43,0.002364,0.796657,110,30,0.1,elu,False,216,...,-1,651494,2018-05-11 03:14:29,1,11,0.21323,0.97596,0.08350,0.97779,0.98450
35,6004926,63,0.003366,0.854811,115,63,0.1,tanh,False,104,...,-1,28508596,2018-05-11 01:13:06,1,17,0.13576,0.98618,0.05317,0.98657,0.98453
34,6001468,76,0.007509,0.823857,126,39,0.3,relu,False,201,...,-1,8242668,2018-05-10 22:14:54,1,4,0.21972,0.97796,0.09099,0.97670,0.98321
33,5800735,43,0.022791,0.794382,137,104,0.2,leakyrelu,False,153,...,-1,26465881,2018-05-08 14:17:37,1,1,15.87176,0.00383,15.90249,0.00250,0.50000
32,5787308,35,0.002751,0.930060,131,248,0.2,relu,True,152,...,-1,33755231,2018-05-08 13:32:15,2,18,0.17508,0.98199,0.06853,0.98281,0.98500


30	0.000754	0.939598	59	320	0.4	relu	True	308
5416730 bst_epoch: 29 loss: 0.2633 - acc: 0.9676 - val_loss: 0.1073 - val_acc: 0.9704 - roc_auc_val: 0.9851
5444132 20ep loss: 0.2763 - acc: 0.9658 - val_loss: 0.1111 - val_acc: 0.9699 - roc_auc_val: 0.9848
5458670: 18ep loss: 0.1904 - acc: 0.9799 - val_loss: 0.0733 - val_acc: 0.9812 - roc_auc_val: 0.9848

nn_record.at[0, 'val_acc'] = 0.9699

nn_record.to_csv('nn_record.csv', index=False, header=False)

In [38]:
selected = list(nn_record.sort_values(by='val_auc', ascending=False)[:11]['tempid'])

In [39]:
selected

[5687118,
 5665676,
 5444132,
 5458670,
 5649200,
 5509928,
 5523788,
 5538577,
 5637753,
 5611185,
 5582707]

In [40]:
import os
selected_files = []
for tempid in selected:
    for filename in os.listdir('NN_SUBS'):
        if (str(tempid) in filename and 'nn' not in filename):
            selected_files.append(filename)
            
print(selected_files)
print(len(selected_files))

['98516_5687118.npy', '98505_5665676.npy', '98098_5444132.npy', '98042_5458670.npy', '98458_5649200.npy', '98456_5509928.npy', '98445_5523788.npy', '98432_5538577.npy', '98428_5637753.npy', '98427_5611185.npy', '98425_5582707.npy']
11


In [41]:
for filename in os.listdir('NN_SUBS'):
    if 'special' in filename:
        selected_files.append(filename)
print(selected_files)
print(len(selected_files))

['98516_5687118.npy', '98505_5665676.npy', '98098_5444132.npy', '98042_5458670.npy', '98458_5649200.npy', '98456_5509928.npy', '98445_5523788.npy', '98432_5538577.npy', '98428_5637753.npy', '98427_5611185.npy', '98425_5582707.npy', '9851_special_5416730.npy']
12


In [42]:
import numpy as np

In [43]:
for i, file in enumerate(selected_files):
    if i == 0:
        preds = np.load('NN_SUBS/'+file)
    else:
        temp = np.load('NN_SUBS/'+file)
        preds += temp
preds = preds/len(selected_files)
print(preds[:2])

[[0.1293739 ]
 [0.02364804]]


In [51]:
predsnpy = np.load('/home/kai/data/kaggle/talkingdata/data/SUBS/FinalSub/lightgbm_best.npy')

In [44]:
ori_path = '/home/kai/data/kaggle/talkingdata/data/'

test_ori = pd.read_csv(ori_path+"test.csv", usecols=['ip','app','device','os', 'channel', 'click_time', 'click_id'])

In [52]:
sub = pd.DataFrame()
sub['click_id'] = test_ori['click_id'].astype('int')

sub['is_attributed'] = predsnpy#preds

In [53]:
sub.shape

(18790469, 2)

In [54]:
sub.tail(2)

,click_id,is_attributed
18790467,18790466,0.014059
18790468,18790468,0.001774


In [55]:
import time
tempid = str(int(time.time()))[3:]
print(tempid) 

5723574


In [56]:
flag = 'lgb_best_'
sub.to_csv('/home/kai/data/kaggle/talkingdata/data/SUBS/FinalSub/{}_{}.csv.gz'.format(flag, tempid), index=False, float_format='%.9f', compression='gzip')

In [49]:
flag = '12AvgBlend'
sub.to_csv('NN_SUBS/nn_{}_{}.csv.gz'.format(flag, tempid), index=False, float_format='%.9f', compression='gzip')